In [19]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [20]:
# Set the path to  training dataset
dataset_path = r"D:\Apil\Broadway Classes\AI CLASSES\AI Project\Binary Classification\train"

# Constants
img_size = (128, 128)
batch_size = 32
epochs = 30 
learning_rate = 0.0001  


In [21]:
# Function to load and preprocess the dataset
def load_and_preprocess_dataset(dataset_path):
    data = []
    labels = []
    
    class_labels = {'cat': 0, 'dog': 1}  # Map class labels to numeric values
    
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize pixel values to be between 0 and 1
            data.append(img)
            labels.append(class_labels[label])

    return np.array(data), np.array(labels)


In [22]:
# Loading and preprocess the dataset
data, labels = load_and_preprocess_dataset(dataset_path)

# Splitingt the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)


In [23]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [24]:
# Using VGG16 as a base model
base_model = VGG16(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

# Freezing the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [25]:
# Create a new model on top
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(1, activation='sigmoid'))


In [26]:
# Compile the model with adjusted learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# Train the model with data augmentation
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), 
                    epochs=epochs, 
                    validation_data=(X_val, y_val))

Epoch 1/30
14/14 [==============================] - 20s 1s/step - loss: 0.6791 - accuracy: 0.6292 - val_loss: 0.4496 - val_accuracy: 0.7857
Epoch 2/30
14/14 [==============================] - 23s 2s/step - loss: 0.4686 - accuracy: 0.7596 - val_loss: 0.3544 - val_accuracy: 0.8214
Epoch 3/30
14/14 [==============================] - 24s 2s/step - loss: 0.4307 - accuracy: 0.7955 - val_loss: 0.3352 - val_accuracy: 0.8214
Epoch 4/30
14/14 [==============================] - 27s 2s/step - loss: 0.3891 - accuracy: 0.8135 - val_loss: 0.3390 - val_accuracy: 0.8214
Epoch 5/30
14/14 [==============================] - 23s 2s/step - loss: 0.3736 - accuracy: 0.8225 - val_loss: 0.3329 - val_accuracy: 0.8393
Epoch 6/30
14/14 [==============================] - 25s 2s/step - loss: 0.3738 - accuracy: 0.8112 - val_loss: 0.3420 - val_accuracy: 0.8393
Epoch 7/30
14/14 [==============================] - 25s 2s/step - loss: 0.3223 - accuracy: 0.8584 - val_loss: 0.3578 - val_accuracy: 0.7857
Epoch 8/30
14/14 [==

In [27]:
# Evaluate the model on the validation set
predictions = (model.predict(X_val) > 0.5).astype("int32")

4/4 [==============================] - 4s 993ms/step


In [28]:
# Print classification report and accuracy
print(classification_report(y_val, predictions))
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

              precision    recall  f1-score   support

           0       0.83      0.86      0.85        64
           1       0.80      0.77      0.79        48

    accuracy                           0.82       112
   macro avg       0.82      0.82      0.82       112
weighted avg       0.82      0.82      0.82       112

Accuracy: 82.14%
